<a href="https://colab.research.google.com/github/Parkhat-archive/Data-Science-Portfolio/blob/master/CHATBOT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 56kB/s 
     |████████████████████████████████| 368kB 35.0MB/s 
     |████████████████████████████████| 3.2MB 37.3MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [ ]:
pip install tflearn

     |████████████████████████████████| 102kB 2.2MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=a9c1aa36c8caf177c804bcaab1a9bacc2b8e082052117a91d3dd8e7cb35ba8ac
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
from google.colab import files
files.upload()


MessageError: ignored

In [ ]:
with open ('intents.json') as json_data:
    data=json.load(json_data)

In [ ]:
try:
  with open('data.pickle', 'rb') as f:
    words, labels, training, output= pickle.load(f)

except:
  words=[]
  labels=[]
  docs_x=[]
  docs_y= []

  for intent in data['intents']:
      for pattern in intent['patterns']:
          wrds=nltk.word_tokenize(pattern)
          words.extend(wrds)
          docs_x.append(wrds)
          docs_y.append(intent['tag'])

          if intent['tag'] not in labels:
              labels.append(intent['tag'])
  words = [stemmer.stem(w.lower()) for w in words if w != '?']
  words = sorted(list(set(words)))

  labels= sorted(list(set(labels)))

  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)

    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    training.append(bag)
    output.append(output_row)

  training=np.array(training)
  output= np.array(output)
  with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

In [ ]:


tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net= tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
  model.load('model.tflearn')
except:
  model.fit(training, output, n_epoch= 1000, batch_size=8, show_metric=True)
  model.save('model.tflearn')

In [ ]:
def bag_of_words(s, words):
  bag=[0 for _ in range(len(words))]

  s_words= nltk.word_tokenize(s)
  s_words= [stemmer.stem(words.lower()) for words in s_words]

  for se in s_words:
    for i,w in enumerate(words):
      if w== se:
        bag[i]=1
  return np.array(bag)

def chat():
  print('Start talking with the bot (Type quit to stop)!')
  while True:
    inp= input('You: ')
    if inp.lower() == 'quit':
      break

    result = model.predict([bag_of_words(inp, words)])[0]
    result_index = np.argmax(result)
    tag = labels[result_index]

    if result[result_index]> 0.7:

      for tg in data['intents']:
        if tg['tag'] == tag:
          responses = tg['responses']
    

      print(random.choice(responses))
    else :
      print('I dont quite understand, Try again')

In [ ]:
chat()